In [ ]:
import h5py, sys
import spimage
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Data access

In [ ]:
runnr = 98
base_path = '/Users/benedikt/data/LCLS/amol3416/pbcv/'
filename_ol1 = base_path + 'amol3416_r%04d_ol1.h5' %runnr
filename_ol3 = base_path + 'amol3416_r%04d_ol3.h5' %runnr
outfolder = base_path + 'phasing/r%04d' %runnr 

# Extract 10 strongest example images

In [ ]:
# Load 10 strongest hits from ouput level 3
with h5py.File(filename_ol3, 'r') as f:    
    hitscore = f['entry_1/result_1/hitscore_litpixel'][:]
    selection = np.sort(np.argsort(hitscore)[-10:])
    data = f['entry_1/data_1/data'][selection,:,:].astype(np.float64)
    gain = f['entry_1/detector_1/gain'][:][selection]
    mask_init = ~f['entry_1/detector_1/mask'][:].astype(np.bool)
    timestamp = f['entry_1/event/timestamp'][:][selection]

In [ ]:
# Parameters
downsampling = 4
saturation_level = 9000

# Loop through the 10 images
for i in range(10):
    
    # Mask out saturated pixels
    mask_saturated = (data[i] < saturation_level)
    mask = mask_init & mask_saturated
    
    # Set background to zero
    intensities = data[i]
    intensities[intensities <= 0.5*gain[i]] = 0.
    
    # Find the center
    x,y = spimage.find_center(intensities,mask, method='blurred', x0=0, y0=0, 
                              dmax=40, threshold=0.5*gain[i], blur_radius=4)
    
    # Crop around the center to 1024x1024
    centery = int(intensities.shape[0]//2+y + 50)
    centerx = int(intensities.shape[1]//2+x + 50)
    intensities_padded = np.pad(intensities, 50, 'constant')
    intensities_cropped = intensities_padded[centery-512:centery+512, centerx-512:centerx+512]
    mask_padded = np.pad(mask, 50, 'constant')
    mask_cropped = mask_padded[centery-512:centery+512, centerx-512:centerx+512]

    # Gain calibration and cropping to 1024 x 1024
    intensities_c = intensities_cropped / gain[i]
    mask_c = mask_cropped
    
    # Downsampling
    intensities_ds, mask_ds = spimage.binImage(intensities_c, downsampling, msk=mask_c, output_binned_mask=True)

    # Photon counting
    intensities_cnt = np.round(intensities_ds)
    mask_cnt = mask_ds.astype(np.bool)
            
    # Write to spimage-type HDF5 file
    ny,nx = intensities_cnt.shape
    img = spimage.sp_image_alloc(nx,ny,1)
    img.phased = 0
    img.detector.image_center[:] =[spimage.center_to_pos(y,nx), spimage.center_to_pos(x,ny),0]
    img.image[:,:] = intensities_cnt
    img.mask[:,:]  = mask_cnt
    spimage.sp_image_write(img,"%s/r%04d_%03i_0.h5" % (outfolder,runnr, i),0)